In [ ]:
import os
import pysam
import yaml
from datetime import datetime

In [ ]:
data_dir = '/home/mwjeong/projects/cwas/data'
vcf_path = f'{data_dir}/input/my_table.hq_dnvs.sorted.cp.vcf.gz'
annot_bed_path = f'{data_dir}/annotate/merge_track.bed.gz'
result_path = f'{data_dir}/input/my_table_hq_dnvs.sorted.cp.annot.vcf'

In [ ]:
# path settings
project_dir = os.path.abspath('..')
annot_conf_path = os.path.join(project_dir, 'conf', 'annotation.yaml')

In [ ]:
# parse the configuration
with open(annot_conf_path) as annot_conf_file:
    annot_path_dict = yaml.safe_load(annot_conf_file)

In [ ]:
# List of bed files
bed_keys = []

for annot_key in annot_path_dict:
    annot_path = annot_path_dict[annot_key]
    if annot_path.endswith('bed') or annot_path.endswith('bed.gz'):
        bed_keys.append(annot_key)

In [ ]:
def get_curr_time() -> str:
    now = datetime.now()
    curr_time = now.strftime('%H:%M:%S %m/%d/%y')
    return curr_time

In [ ]:
# Custom annotation
chroms = [f'chr{n}' for n in range(1, 23)]

with pysam.TabixFile(vcf_path) as vcf_file, pysam.TabixFile(annot_bed_path) as bed_file, open(result_path, 'w') as outfile:
    print(f'##INFO=<ID=ANNOT,LIST={"|".join(bed_keys)}>', file=outfile)
    print('#CHROM', 'POS', 'ID', 'REF', 'ALT', 'QUAL', 'FILTER', 'INFO', sep='\t', file=outfile)

    for chrom in chroms:
        print(f'[{get_curr_time()}, Progress] {chrom}')
        var_iter = vcf_file.fetch(chrom, parser=pysam.asTuple())
        bed_iter = bed_file.fetch(chrom, parser=pysam.asTuple())
        variant = next(var_iter, None)
        bed = next(bed_iter, None)

        while variant is not None and bed is not None:
            var_pos = int(variant[1]) - 1
            bed_start = int(bed[1])
            bed_end = int(bed[2])

            if var_pos >= bed_end:
                bed = next(bed_iter, None)
            else:
                if var_pos < bed_start:
                    annot_int = 0
                else:
                    annot_int = int(bed[3])

                print(str(variant) + f';ANNOT={annot_int}', file=outfile)
                variant = next(var_iter, None)

In [ ]:
# VEP
vep_script = '/home/mwjeong/projects/cwas/src/run_vep.py'
vep_result_path = result_path.replace('.vcf', '.vep.vcf')
cmd = f'{vep_script} -i {result_path} -o {vep_result_path};'
print(f'[{get_curr_time()}, Progress] VEP')
os.system(cmd)
print(f'[{get_curr_time()}, Progress] Done')